In [1]:
import torch
from torch.autograd import Variable
from torch.nn import Parameter
from torch.utils.data import DataLoader

from torchvision.datasets import mnist

from torch import nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os,time

In [9]:
train_set=mnist.MNIST('./data/',train=True)
test_set=mnist.MNIST('./data/',train=False)

In [14]:
def data_df(x):
    x=np.array(x)/255
    x=(x-0.5)/0.5
    x=x.reshape((-1,))
    x=torch.Tensor(x).float()
    return x

In [16]:
train_set=mnist.MNIST('./data/',train=True,transform=data_df)
test_set=mnist.MNIST('./data/',train=False,transform=data_df)

In [20]:
train_data=DataLoader(train_set,batch_size=64,shuffle=True)
test_data=DataLoader(test_set,batch_size=128,shuffle=False)

In [26]:
net=nn.Sequential(
    nn.Linear(784,400),
    nn.ReLU(),
    nn.Linear(400,200),
    nn.ReLU(),
    nn.Linear(200,100),
    nn.ReLU(),
    nn.Linear(100,10)
)

In [28]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=.1)

In [30]:
losses=[]
acces=[]
eval_losses=[]
eval_acces=[]

for i in range(20):
    train_loss=0
    train_acc=0
    net.train()
    
    for im,label in train_data:
        out=net(im)
        loss=criterion(out,label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.data.numpy()
        _,pred=out.max(1)
        acc=(pred==label).float().sum()/im.shape[0]
        train_acc+=acc
    losses.append(train_loss)
    acces.append(train_acc/len(train_data))
    
    eval_loss=0
    eval_acc=0
    net.eval()
    
    for im,label in test_data:
        out=net(im)
        loss=criterion(out,label)
        
        eval_loss+=loss.data.numpy()
        
        _,pred=out.max(1)
        acc=(pred==label).float().sum()/im.shape[0]
        eval_acc+=acc
        
    eval_losses.append(eval_loss/len(test_data))
    eval_acces.append(eval_acc/len(test_data))
    
    print('epoch:{} train_loss:{} test_loss:{} train_acc:{} test_acc:{}'.format(i+1,train_loss/len(train_data),eval_loss/len(test_data),train_acc/len(train_data),eval_acc/len(test_data)))

epoch:1 train_loss:159.3747474476695 test_loss:16.50667974539101 train_acc:0.946911633014679 test_acc:0.9358188509941101
epoch:2 train_loss:112.87833075970411 test_loss:32.60004784911871 train_acc:0.962619960308075 test_acc:0.8525514006614685
epoch:3 train_loss:88.16598168760538 test_loss:7.459538152441382 train_acc:0.970565676689148 test_acc:0.9712222814559937
epoch:4 train_loss:69.83614106476307 test_loss:6.245580837130547 train_acc:0.9765791296958923 test_acc:0.9758702516555786
epoch:5 train_loss:58.382567312568426 test_loss:11.839364130049944 train_acc:0.9799273610115051 test_acc:0.9551028609275818
epoch:6 train_loss:50.06894477084279 test_loss:31.213485591113567 train_acc:0.9831756353378296 test_acc:0.9075356125831604
epoch:7 train_loss:42.63823311962187 test_loss:6.155238473787904 train_acc:0.9849913120269775 test_acc:0.9757713675498962
epoch:8 train_loss:37.24673214368522 test_loss:7.030034746974707 train_acc:0.9869070053100586 test_acc:0.9755735993385315
epoch:9 train_loss:32.3